<a href="https://colab.research.google.com/github/kryby/Best-README-Template/blob/master/mental_health_work_place_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import os
import sys
import io
import warnings
warnings.filterwarnings('ignore')

# --- Display settings ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Auxiliary function to search columns by keywords
def find_cols(df,keywords, name):
        cols = []
        for kw in keywords:
            found = [col for col in df.columns if kw.lower() in col.lower()]
            cols.extend(found)
        return list(set(cols))  # Eliminar duplicados

# Load data with automatic detection and perform initial scan
def load_and_explore_data(filename='mental_health.csv'):


    print("STEP 1: DATA LOADING AND EXPLORATION")

    if os.path.exists(filename):
        df = pd.read_csv(filename)
        print(f"'{filename}' Loaded.")
    else:
        raise FileNotFoundError("File not found. Ensure that the input file mental_health.csv is in the same directory as the script.")

    #Sanitize column names
    # original columns names backup
    original_columns = df.columns.tolist()


    df.columns = (df.columns
                  .str.strip()
                  .str.replace('\s+', ' ', regex=True)
                  .str.replace('?', '', regex=False)
                  .str.replace(':', '', regex=False)
                  .str.replace('"', '', regex=False)
                  .str.replace('  ', ' ')
                  .str.strip())

    print(f"\n✓ {len(df.columns)} columnas loaded.")
    print(f"✓ {len(df)} rows loaded.")

    # Initial exploration

    print("\n" + "-"*80)
    print("Basic descriptive statistics of the file")
    print("-"*80)

    #Dataset general information
    print("\nDataset info:")
    buffer = io.StringIO()
    df.info(buf=buffer)
    print(buffer.getvalue())
    buffer.close()

    # Top rows
    print("\n First 3 rows:")
    print(df.head(3).to_string())

    # Null values by column
    null_counts = df.isnull().sum()
    null_percentages = (df.isnull().sum() / len(df)) * 100

    print("\n Null values by column:")
    null_df = pd.DataFrame({
        'Nulls': null_counts,
        'Percent': null_percentages
    }).sort_values('Percent', ascending=False)
    print(null_df.to_string())

    #Data type
    print("\n Data types:")
    print(df.dtypes.value_counts().to_string())

    #
    dtypes_count = df.dtypes.value_counts()
    for dtype, count in dtypes_count.items():
        print(f"   {dtype}: {count} columns")

    # Columns by category (manual identification)
    print("\n Identified column categories:")




    # Search for relevant columns
    support_keywords = ['benefits', 'options', 'resources', 'anonymity',
                        'formally discussed', 'mental health']
    stigma_keywords = ['consequences', 'observed','observations',
                       'negative consequences','medical leave','negatively',
                       'reveal','career','share','unsupportive','interferes']
    openness_keywords = ['coworkers', 'supervisor', 'seriously',
                         'medical leave', 'wellness','Why or why not']
    treatment_keywords = ['sought treatment', 'mental health professional',
                          'diagnosed', 'condition', 'disorder','health issue']
    work_keywords = ['position', 'remote', 'tech company', 'role', 'employees',
                     'employed','employer','work remotely','clients',
                     'business contacts','productivity','work']
    demo_keywords = ['what is your age', 'gender', 'country', 'state',
                     'live', 'work',
                     'us state','family history']

    support_cols = find_cols(df, support_keywords, 'support')
    stigma_cols = find_cols(df,stigma_keywords, 'stigma')
    openness_cols = find_cols(df,openness_keywords, 'openness')
    treatment_cols = find_cols(df,treatment_keywords, 'treatment')
    work_cols = find_cols(df,work_keywords, 'work')
    demo_cols = find_cols(df,demo_keywords, 'demo')

    print(f"\n Mental health support: {len(support_cols)} columns")
    print(f"\n Work stigma: {len(stigma_cols)} columns")
    print(f"\n Organisational openness: {len(openness_cols)} columns")
    print(f"\n Diag/treatment{len(treatment_cols)} columns")
    print(f"\n Work: {len(work_cols)} columns")
    print(f"\n Demographics: {len(demo_cols)} columns")

    # Columns by category
    print("\nColumns by category")
    if support_cols:
        print(f"\n Support: {support_cols}")
    if stigma_cols:
        print(f"\n Stigma: {stigma_cols}")
    if openness_cols:
        print(f"\n Openness: {openness_cols}")

    #Estadísticas de columnas numéricas (edad)
    print("\n Age column analysis:")
    age_col = next((col for col in df.columns if 'what is your age' in col.lower()), None)
    if age_col:
        print(f"\n Identified column: '{age_col}'")

        # Convert to numbers
        age_numeric = pd.to_numeric(df[age_col], errors='coerce')
        print(f"\n   Age statistics:")
        print(f"   Valid values: {age_numeric.notna().sum()} out of {len(df)}")
        print(f"   Mean: {age_numeric.mean():.1f}")
        print(f"   Median: {age_numeric.median():.1f}")
        print(f"   Min: {age_numeric.min():.0f}")
        print(f"   Max: {age_numeric.max():.0f}")

        # Outliers detection
        outliers = age_numeric[(age_numeric < 18) | (age_numeric > 75)].dropna()
        if len(outliers) > 0:
            print("-"*80)
            print(f"OUTLIERS DETECTED")
            print("-"*80)
            print(f"{len(outliers)} values out of range 18-75")
            print(f"      outliers: {outliers.unique()}")
    else:
        print("Age column not found.")

    # Null values by category
    print("\nNull values by category:")
    print(f"   • Support: {df[support_cols].isnull().sum().sum() if support_cols else 0} null cells")
    print(f"   • Stigma: {df[stigma_cols].isnull().sum().sum() if stigma_cols else 0} null cells")
    print(f"   • Openness: {df[openness_cols].isnull().sum().sum() if openness_cols else 0} null cells")
    print(f"   • Treatment: {df[treatment_cols].isnull().sum().sum() if treatment_cols else 0} null cells")

    print("\n" + "="*80)
    print("STEP 1 COMPLETED")
    print("="*80)

    return df, {
        'original_columns': original_columns,
        'support_cols': support_cols,
        'stigma_cols': stigma_cols,
        'openness_cols': openness_cols,
        'treatment_cols': treatment_cols,
        'work_cols': work_cols,
        'demo_cols': demo_cols
    }

# MAIN

if __name__ == "__main__":
    # Load dadta
    df, column_categories = load_and_explore_data("mental_health.csv")

    # Mostrar resumen de categorías encontradas
    print("\n" + "="*80)
    print("COLUMNS BY CATEGORY")
    print("="*80)
    for category, cols in column_categories.items():
        if category != 'original_columns':
            print(f"\n{category.upper()}: {len(cols)} columns")
            if len(cols) > 0:
                print(f"   {cols}")  # Mostrar primeras 5

    # Guardar el DataFrame limpio y las categorías para el siguiente paso
    df.to_csv('mental_health_clean_headers.csv', index=False)

    # Guardar las categorías en un archivo para referencia
    import json
    categories_serializable = {k: v for k, v in column_categories.items()
                              if k != 'original_columns'}
    with open('column_categories.json', 'w') as f:
        json.dump(categories_serializable, f, indent=2, default=list)

    print("\n💾 DataFrame guardado como 'mental_health_clean_headers.csv'")
    print("💾 Categorías guardadas como 'column_categories.json'")

STEP 1: DATA LOADING AND EXPLORATION
'mental_health.csv' Loaded.

✓ 62 columnas loaded.
✓ 1433 rows loaded.

--------------------------------------------------------------------------------
Basic descriptive statistics of the file
--------------------------------------------------------------------------------

Dataset info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1433 entries, 0 to 1432
Data columns (total 62 columns):
 #   Column                                                                                                                                                                           Non-Null Count  Dtype  
---  ------                                                                                                                                                                           --------------  -----  
 0   Are you self-employed                                                                                                                                

In [7]:
import pandas as pd
import numpy as np
import json
from collections import Counter
import re

#Build a manually corrected mapping dictionary based on analysis.
def build_corrected_mapping():

    # base mapping
    response_map = {
        # --- NEGATIVE---
        'no': 0,
        'never': 0,
        'not applicable to me': 0,
        'not eligible for coverage / n/a': 0,
        'none did': 0,
        'no, none did': 0,
        'none of them': 0,
        'no, at none of my previous employers': 0,
        'n/a (not currently aware)': 0,
        'no, i only became aware later': 0,
        'no, because it doesn\'t matter': 0,
        'no, because it would impact me negatively': 0,
        'no, i don\'t know any': 0,
        'not relevant': 0,
        'would not': 0,
        'never discuss': 0,
        'not eligible for coverage / n/a': 0,
        'not eligible for coverage': 0,

        # --- NEUTRAL ---
        'maybe': 1,
        'maybe/not sure': 1,
        'i don\'t know': 1,
        'i am not sure': 1,
        'not sure': 1,
        'unsure': 1,
        'neutral': 1,
        'somewhat difficult': 1,
        'neither easy nor difficult': 1,
        'some did': 1,
        'some of them': 1,
        'sometimes, if it comes up': 1,
        'i don\'t know (sorry)': 1,
        'i was aware of some': 1,
        'i know some': 1,
        'rarely': 1,
        'sometimes': 2,
        'i don\'t know': 1,
        'i am not sure': 1,

        # --- POSITIVE---
        'yes': 2,
        'very open': 3,
        'somewhat open': 2,
        'very easy': 3,
        'somewhat easy': 2,
        'yes, always': 2,
        'they all did': 2,
        'yes, they all did': 2,
        'yes, i was aware of all of them': 2,
        'yes, at all of my previous employers': 2,
        'yes, i experienced': 2,
        'yes, i observed': 2,
        'yes, all of them': 2,
        'yes, i know several': 2,
        'always': 3,
        'often': 3,
        'very frequently': 3,

        # --- Frequencies (0-3) ---
        'never': 0,
        'rarely': 1,
        'sometimes': 2,
        'often': 3,
        'always': 3,

        # --- EASY TO REQUEST LEAVE (0-3) ---
        'very difficult': 0,
        'somewhat difficult': 1,
        'neither easy nor difficult': 1,
        'somewhat easy': 2,
        'very easy': 3,

        # --- COMPANY SIZE (ORDINAL) ---
        '1-5': 1,
        '6-25': 2,
        '26-100': 3,
        '100-500': 4,
        '500-1000': 5,
        'more than 1000': 6,

        # --- REMOTE WORK (0-2)---
        'never': 0,
        'rarely': 0,
        'sometimes': 1,
        'often': 2,
        'always': 2,

        # --- DIAG RESPONSES ---
        '0': 0,
        '1': 1,

        # --- TIME PERCENTAGE AFFECTED ---
        '1-25%': 1,
        '26-50%': 2,
        '51-75%': 3,
        '76-100%': 4,
    }

    return response_map

def clean_age_manual(df):

    age_col = next((col for col in df.columns if 'what is your age' in col.lower()), None)

    # --- 2. MOSTRAR VALORES ORIGINALES ---
    print(f"\n📊 Procesando columna: '{age_col}'")

    # Mostrar valores únicos antes de limpiar
    unique_ages = df[age_col].dropna().unique()
    unique_ages_sample = sorted(unique_ages)[:20] if len(unique_ages) > 20 else sorted(unique_ages)
    print(f"\n📋 Valores únicos de edad (primeros 20):")
    for i, val in enumerate(unique_ages_sample, 1):
        print(f"   {i:2d}. '{val}'")
    print(f"   ... y {len(unique_ages) - 20} valores más" if len(unique_ages) > 20 else "")

    # --- 3. CONVERTIR A NUMÉRICO ---
    df['age_clean'] = pd.to_numeric(df[age_col], errors='coerce')

    # Estadísticas antes de limpiar
    valid_before = df['age_clean'].notna().sum()
    print(f"\n📊 Estadísticas ANTES de limpiar:")
    print(f"   • Valores válidos: {valid_before} de {len(df)} ({valid_before/len(df)*100:.1f}%)")
    print(f"   • Media: {df['age_clean'].mean():.1f}")
    print(f"   • Mediana: {df['age_clean'].median():.1f}")
    print(f"   • Mínimo: {df['age_clean'].min():.0f}")
    print(f"   • Máximo: {df['age_clean'].max():.0f}")

    # --- 4. FILTRAR EDADES RAZONABLES (18-75) ---
    outliers_before = df['age_clean'].isna().sum()

    # Marcar outliers
    mask_too_young = df['age_clean'] < 18
    mask_too_old = df['age_clean'] > 75

    df.loc[mask_too_young, 'age_clean'] = np.nan
    df.loc[mask_too_old, 'age_clean'] = np.nan

    outliers_removed = mask_too_young.sum() + mask_too_old.sum()

    if outliers_removed > 0:
        print(f"\n⚠ Outliers eliminados: {outliers_removed}")
        young_outliers = df.loc[mask_too_young, age_col].unique() if mask_too_young.any() else []
        old_outliers = df.loc[mask_too_old, age_col].unique() if mask_too_old.any() else []

        if len(young_outliers) > 0:
            print(f"   • Edades < 18: {sorted(young_outliers)[:5]}")
        if len(old_outliers) > 0:
            print(f"   • Edades > 80: {sorted(old_outliers)[:5]}")

    # --- 5. IMPUTAR CON MEDIANA ---
    median_age = df['age_clean'].median()
    df['age_clean'] = df['age_clean'].fillna(median_age)

    print(f"\n📊 Estadísticas DESPUÉS de limpiar:")
    print(f"   • Mediana usada para imputación: {median_age:.0f}")
    print(f"   • Valores válidos: {df['age_clean'].notna().sum()} de {len(df)}")
    print(f"   • Media: {df['age_clean'].mean():.1f}")
    print(f"   • Mínimo: {df['age_clean'].min():.0f}")
    print(f"   • Máximo: {df['age_clean'].max():.0f}")

    # --- 6. VERIFICACIÓN FINAL ---
    print(f"\n✅ VERIFICACIÓN: Edad limpiada correctamente")
    print(f"   • Columna original: '{age_col}'")
    print(f"   • Columna creada: 'age_clean'")

    return df, age_col

def clean_gender_corrected(text):
    """Limpieza ROBUSTA de género con categorías correctas"""
    if pd.isna(text):
        return 2

    t = str(text).lower().strip()

    # 1. MASCULINO (0)
    male_terms = ['male', 'man', 'm', 'cis male', 'cis man', 'masculine', 'mail', 'maile', 'man ', ' cis male']
    if any(term in t for term in male_terms):
        return 0

    # 2. FEMENINO (1)
    female_terms = ['female', 'woman', 'f', 'cis female', 'cis woman', 'feminine', 'fem', 'woman ']
    if any(term in t for term in female_terms):
        return 1

    # 3. OTROS / NO BINARIO (2)
    return 2

def parse_multi_choice(text, separator='|'):
    """Convierte respuestas múltiples (separadas por |) en lista"""
    if pd.isna(text):
        return []
    return [item.strip() for item in str(text).split(separator)]

def extract_top_roles(series, top_n=12):
    """Extrae los roles más comunes de la columna de posición laboral"""
    all_roles = []
    for val in series.dropna():
        roles = parse_multi_choice(val)
        all_roles.extend(roles)

    role_counts = Counter(all_roles)
    top_roles = [role for role, count in role_counts.most_common(top_n)]

    print(f"\n📊 Top {top_n} roles laborales:")
    for i, role in enumerate(top_roles, 1):
        pct = (role_counts[role] / len(series.dropna())) * 100
        print(f"   {i:2d}. {role[:35]:35} : {role_counts[role]:3d} ({pct:4.1f}%)")

    return top_roles

def create_role_features(df, role_col, top_roles):
    """Crea features binarias para los roles más comunes"""
    role_features = []

    for role in top_roles:
        # Crear nombre de columna seguro
        col_name = role.lower()
        col_name = re.sub(r'[^a-z0-9\s]', '', col_name)
        col_name = col_name.replace(' ', '_')
        col_name = f'role_{col_name}'[:50]

        df[col_name] = df[role_col].apply(
            lambda x: 1 if not pd.isna(x) and role.lower() in str(x).lower() else 0
        )
        role_features.append(col_name)

    return df, role_features

def extract_conditions(text):
    """Extrae condiciones de salud mental de texto con múltiples valores"""
    conditions = parse_multi_choice(text)

    categories = {
        'anxiety': 0,
        'depression': 0,
        'adhd': 0,
        'ptsd': 0,
        'ocd': 0,
        'eating_disorder': 0,
        'personality': 0,
        'substance': 0,
        'addictive': 0,
        'bipolar': 0,
        'psychotic': 0,
        'stress': 0,
        'dissociative': 0,
        'autism': 0,
    }

    keywords = {
        'anxiety': ['anxiety', 'panic', 'phobia', 'gad'],
        'depression': ['depression', 'dysthymia', 'mood disorder'],
        'adhd': ['adhd', 'add', 'attention deficit', 'hyperactivity'],
        'ptsd': ['ptsd', 'post traumatic', 'post-traumatic'],
        'ocd': ['ocd', 'obsessive', 'compulsive'],
        'eating_disorder': ['anorexia', 'bulimia', 'eating disorder'],
        'personality': ['borderline', 'antisocial', 'paranoid', 'personality'],
        'substance': ['substance', 'alcohol', 'drug'],
        'addictive': ['addictive', 'addiction'],
        'bipolar': ['bipolar'],
        'psychotic': ['schizo', 'psychotic', 'psychosis'],
        'stress': ['stress', 'burnout', 'adjustment'],
        'dissociative': ['dissociative', 'depersonalization'],
        'autism': ['autism', 'autistic', 'asperger', 'asd'],
    }

    for condition in conditions:
        cond_lower = condition.lower()
        for category, kw_list in keywords.items():
            if any(kw in cond_lower for kw in kw_list):
                categories[category] = 1

    return pd.Series(categories)

# ============================================
# EJECUCIÓN PRINCIPAL
# ============================================

if __name__ == "__main__":
    print("="*80)
    print("🚀 FASE 2.3: LIMPIEZA FINAL CON CORRECCIÓN MANUAL DE EDAD")
    print("="*80)

    # 1. Cargar datos
    df = pd.read_csv('mental_health_clean_headers.csv')
    print(f"\n✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")

    # 2. Construir diccionario de mapeo
    response_map = build_corrected_mapping()
    print(f"\n📚 Diccionario base: {len(response_map)} entradas")

    # 3. LIMPIEZA DE EDAD - CORRECCIÓN MANUAL
    df, age_col = clean_age_manual(df)

    # 4. LIMPIEZA DE GÉNERO
    print("\n" + "-"*80)
    print("📊 LIMPIEZA DE GÉNERO")
    print("-"*80)

    gender_col = next((c for c in df.columns if 'gender' in c.lower()), None)
    if gender_col:
        print(f"\n▶ Columna de género: '{gender_col}'")
        df['gender_clean'] = df[gender_col].apply(clean_gender_corrected)

        gender_dist = df['gender_clean'].value_counts().sort_index()
        print(f"\nDistribución de género:")
        for val, count in gender_dist.items():
            label = {0: 'Masculino', 1: 'Femenino', 2: 'Otro/No binario'}.get(val, 'Unknown')
            print(f"   {val} ({label}): {count:4d} ({count/len(df)*100:5.1f}%)")

    # 5. ANÁLISIS DE POSICIÓN LABORAL
    print("\n" + "-"*80)
    print("📊 ANÁLISIS DE POSICIÓN LABORAL")
    print("-"*80)

    role_col = next((c for c in df.columns if 'work position' in c.lower()), None)
    if role_col:
        print(f"\n▶ Columna de posición: '{role_col[:60]}...'")

        null_pct = df[role_col].isnull().mean() * 100
        print(f"   Valores nulos: {null_pct:.1f}%")

        if null_pct < 90:
            top_roles = extract_top_roles(df[role_col], top_n=12)
            df, role_features = create_role_features(df, role_col, top_roles)
            print(f"\n✅ Creadas {len(role_features)} features de roles")

    # 6. ANÁLISIS DE CONDICIONES
    print("\n" + "-"*80)
    print("📊 ANÁLISIS DE CONDICIONES DE SALUD MENTAL")
    print("-"*80)

    # Diagnósticos
    diag_cond_col = next((c for c in df.columns if 'condition(s) were you diagnosed' in c.lower()), None)
    if diag_cond_col:
        print(f"\n▶ Columna de diagnósticos encontrada")
        condition_features = df[diag_cond_col].apply(extract_conditions)
        condition_features.columns = [f'diag_{col}' for col in condition_features.columns]
        df = pd.concat([df, condition_features], axis=1)

        print(f"✅ Creadas {len(condition_features.columns)} features de diagnósticos")

    # Sospechas
    suspect_cond_col = next((c for c in df.columns if 'condition(s) do you believe' in c.lower()), None)
    if suspect_cond_col:
        print(f"\n▶ Columna de sospechas encontrada")
        suspect_features = df[suspect_cond_col].apply(extract_conditions)
        suspect_features.columns = [f'suspect_{col}' for col in suspect_features.columns]
        df = pd.concat([df, suspect_features], axis=1)

        print(f"✅ Creadas {len(suspect_features.columns)} features de sospechas")

    # 7. NEUROFLAG COMBINADO
    print("\n" + "-"*80)
    print("🧠 FEATURE DE NEURODIVERGENCIA")
    print("-"*80)

    neuro_columns = [col for col in df.columns if col.startswith('diag_') or col.startswith('suspect_')]
    if neuro_columns:
        df['neuro_flag'] = df[neuro_columns].any(axis=1).astype(int)
        print(f"\n✅ Neuroflag creado:")
        print(f"   • Personas con condición: {df['neuro_flag'].sum():4d} ({df['neuro_flag'].sum()/len(df)*100:5.1f}%)")
        print(f"   • Personas sin condición:  {len(df) - df['neuro_flag'].sum():4d} ({(1-df['neuro_flag'].sum()/len(df))*100:5.1f}%)")

    # 8. GUARDAR RESULTADOS FINALES
    print("\n" + "="*80)
    print("💾 GUARDANDO RESULTADOS FINALES")
    print("="*80)

    # Guardar diccionario
    with open('response_map_final.json', 'w') as f:
        json.dump({str(k): v for k, v in response_map.items()}, f, indent=2)

    # Guardar DataFrame
    output_file = 'mental_health_ready_for_modeling.csv'
    df.to_csv(output_file, index=False)

    print(f"\n✅ DATOS LISTOS PARA MODELADO:")
    print(f"   • Archivo: '{output_file}'")
    print(f"   • Filas: {df.shape[0]:,}")
    print(f"   • Columnas: {df.shape[1]}")

    # Resumen de features creadas
    print(f"\n📋 RESUMEN DE FEATURES CREADAS:")
    print(f"   • age_clean: Edad limpia")
    print(f"   • gender_clean: Género codificado (0=M,1=F,2=O)")
    print(f"   • neuro_flag: Neurodivergencia (0/1)")

    role_count = len([c for c in df.columns if c.startswith('role_')])
    if role_count > 0:
        print(f"   • role_*: {role_count} features de roles laborales")

    diag_count = len([c for c in df.columns if c.startswith('diag_')])
    if diag_count > 0:
        print(f"   • diag_*: {diag_count} features de diagnósticos")

    suspect_count = len([c for c in df.columns if c.startswith('suspect_')])
    if suspect_count > 0:
        print(f"   • suspect_*: {suspect_count} features de sospechas")

    print("\n" + "="*80)
    print("🎉 FASE 2.3 COMPLETADA - DATOS PREPARADOS PARA MODELADO")
    print("="*80)

🚀 FASE 2.3: LIMPIEZA FINAL CON CORRECCIÓN MANUAL DE EDAD

✓ Datos cargados: 1433 filas, 62 columnas

📚 Diccionario base: 64 entradas

📊 Procesando columna: 'What is your age'

📋 Valores únicos de edad (primeros 20):
    1. '3'
    2. '15'
    3. '17'
    4. '19'
    5. '20'
    6. '21'
    7. '22'
    8. '23'
    9. '24'
   10. '25'
   11. '26'
   12. '27'
   13. '28'
   14. '29'
   15. '30'
   16. '31'
   17. '32'
   18. '33'
   19. '34'
   20. '35'
   ... y 33 valores más

📊 Estadísticas ANTES de limpiar:
   • Valores válidos: 1433 de 1433 (100.0%)
   • Media: 34.3
   • Mediana: 33.0
   • Mínimo: 3
   • Máximo: 323

⚠ Outliers eliminados: 5
   • Edades < 18: [np.int64(3), np.int64(15), np.int64(17)]
   • Edades > 80: [np.int64(99), np.int64(323)]

📊 Estadísticas DESPUÉS de limpiar:
   • Mediana usada para imputación: 33
   • Valores válidos: 1433 de 1433
   • Media: 34.1
   • Mínimo: 19
   • Máximo: 74

✅ VERIFICACIÓN: Edad limpiada correctamente
   • Columna original: 'What is your 

In [8]:
# ============================================
# MENTAL HEALTH PREDICTIVE SYSTEM - STEP 3
# CÁLCULO DE ÍNDICES Y CORRELACIONES (SPEARMAN)
# ============================================

import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

# Configuración
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

def load_clean_data():
    """Carga los datos limpios del paso anterior"""
    df = pd.read_csv('mental_health_ready_for_modeling.csv')

    # Cargar diccionario de mapeo
    with open('response_map_final.json', 'r') as f:
        response_map = json.load(f)

    return df, response_map

def identify_index_columns(df):
    """
    IDENTIFICA LAS COLUMNAS EXACTAS PARA CADA ÍNDICE
    Carga las categorías de columnas desde el archivo JSON.
    """

    print("\nLoading column categories from 'column_categories.json'...")
    try:
        with open('column_categories.json', 'r') as f:
            column_categories = json.load(f)
    except FileNotFoundError:
        print("Error: 'column_categories.json' not found. Please ensure STEP 1 was executed correctly.")
        return [], [], [] # Return empty lists to prevent further errors

    support_cols = column_categories.get('support_cols', [])
    stigma_cols = column_categories.get('stigma_cols', [])
    openness_cols = column_categories.get('openness_cols', [])

    # Verify that identified columns actually exist in the DataFrame
    support_cols = [col for col in support_cols if col in df.columns]
    stigma_cols = [col for col in stigma_cols if col in df.columns]
    openness_cols = [col for col in openness_cols if col in df.columns]

    if not support_cols:
        print("Warning: No 'support_cols' found or matching in DataFrame.")
    if not stigma_cols:
        print("Warning: No 'stigma_cols' found or matching in DataFrame.")
    if not openness_cols:
        print("Warning: No 'openness_cols' found or matching in DataFrame.")

    return support_cols, stigma_cols, openness_cols

def map_responses_safe(df, cols, response_map):
    """
    MAPEA RESPUESTAS DE FORMA SEGURA - maneja valores no mapeados
    """
    mapped_df = pd.DataFrame(index=df.index)

    for col in cols:
        if col not in df.columns:
            continue

        # Valores originales
        values = df[col].astype(str).str.lower().str.strip()

        # Mapear usando el diccionario
        mapped_values = values.map(response_map)

        # Identificar valores no mapeados
        unmapped = values[~values.isin(response_map.keys()) & ~values.isna()].unique()

        if len(unmapped) > 0:
            print(f"  ⚠ Columna '{col[:50]}...' tiene {len(unmapped)} valores no mapeados")
            print(f"     Ejemplos: {unmapped[:3]}")

            # Asignar valor por defecto (1 = neutral)
            for val in unmapped:
                if val not in response_map:
                    response_map[val] = 1

        # Re-mapear con diccionario actualizado
        mapped_values = values.map(response_map)
        mapped_df[col] = pd.to_numeric(mapped_values, errors='coerce')

    return mapped_df, response_map

def calculate_spearman_correlations(df, cols, label):
    """
    CALCULA CORRELACIONES DE SPEARMAN y obtiene TOP 5
    Spearman es mejor para datos ordinales
    """
    print(f"\n{'='*60}")
    print(f"📊 CORRELACIONES: {label}")
    print(f"{'='*60}")

    # Mapear respuestas
    df_mapped, _ = map_responses_safe(df, cols, response_map.copy())

    # Eliminar columnas sin datos
    df_mapped = df_mapped.dropna(axis=1, how='all')

    if len(df_mapped.columns) < 2:
        print(f"❌ No hay suficientes columnas para correlación")
        return []

    print(f"\n📋 Columnas analizadas: {list(df_mapped.columns)}")

    # Calcular correlación de Spearman
    corr_matrix = df_mapped.corr(method='spearman').abs()

    # Obtener triángulo superior
    mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    pairs = corr_matrix.where(mask).stack().sort_values(ascending=False)

    # Top 5
    top5 = pairs.head(5)

    print(f"\n🏆 TOP 5 CORRELACIONES:")
    for i, ((col1, col2), val) in enumerate(top5.items(), 1):
        # Acortar nombres para mejor visualización
        c1 = col1[:40] + '...' if len(col1) > 40 else col1
        c2 = col2[:40] + '...' if len(col2) > 40 else col2
        print(f"   {i}. {c1}")
        print(f"      vs {c2}")
        print(f"      Correlación: {val:.4f}\n")

    return [(col1, col2, val) for (col1, col2), val in top5.items()]

def calculate_index_scores(df, support_cols, stigma_cols, openness_cols, response_map):
    """
    CALCULA LOS ÍNDICES (SUPPORT, STIGMA, OPENNESS)
    """
    print(f"\n{'='*60}")
    print(f"📊 CÁLCULO DE ÍNDICES")
    print(f"{'='*60}")

    # Support Index (mayor = mejor soporte)
    print(f"\n🔵 SUPPORT INDEX - {len(support_cols)} columnas")
    support_mapped, _ = map_responses_safe(df, support_cols, response_map)
    df['support_index'] = support_mapped.mean(axis=1, skipna=True)
    print(f"   ✓ Support Index creado")
    print(f"     Media: {df['support_index'].mean():.3f}")
    print(f"     Std:   {df['support_index'].std():.3f}")
    print(f"     Rango: [{df['support_index'].min():.3f}, {df['support_index'].max():.3f}]")

    # Stigma Index (mayor = más estigma)
    print(f"\n🔴 STIGMA INDEX - {len(stigma_cols)} columnas")
    stigma_mapped, _ = map_responses_safe(df, stigma_cols, response_map)
    df['stigma_index'] = stigma_mapped.mean(axis=1, skipna=True)
    print(f"   ✓ Stigma Index creado")
    print(f"     Media: {df['stigma_index'].mean():.3f}")
    print(f"     Std:   {df['stigma_index'].std():.3f}")
    print(f"     Rango: [{df['stigma_index'].min():.3f}, {df['stigma_index'].max():.3f}]")

    # Openness Index (mayor = más apertura)
    print(f"\n🟢 OPENNESS INDEX - {len(openness_cols)} columnas")
    openness_mapped, _ = map_responses_safe(df, openness_cols, response_map)
    df['openness_index'] = openness_mapped.mean(axis=1, skipna=True)
    print(f"   ✓ Openness Index creado")
    print(f"     Media: {df['openness_index'].mean():.3f}")
    print(f"     Std:   {df['openness_index'].std():.3f}")
    print(f"     Rango: [{df['openness_index'].min():.3f}, {df['openness_index'].max():.3f}]")

    return df

def create_target_variables(df):
    """
    CREA LAS VARIABLES TARGET PARA LOS MODELOS
    """
    print(f"\n{'='*60}")
    print(f"🎯 CREACIÓN DE VARIABLES TARGET")
    print(f"{'='*60}")

    # Target 1: Do you currently have a mental health disorder?
    diag_col = next((c for c in df.columns if 'currently have a mental health disorder' in c.lower()), None)
    if diag_col:
        print(f"\n▶ Target 1: '{diag_col[:60]}...'", df[diag_col].head().to_string())

        # Mapear: Yes=1, No/Maybe=0
        df['target_diag'] = df[diag_col].map({'Yes': 1, 'No': 0, 'Maybe': 0}).fillna(0)

        print(f"   Distribución:")
        print(f"     • Diagnosticados: {df['target_diag'].sum():4d} ({df['target_diag'].sum()/len(df)*100:5.1f}%)者に {df['target_diag'].sum():4d} ({(1-df['target_diag'].sum()/len(df))*100:5.1f}%)")

    # Target 2: Have you ever sought treatment?
    treat_col = next((c for c in df.columns if 'sought treatment' in c.lower()), None)
    if treat_col:
        print(f"\n▶ Target 2: '{treat_col[:60]}...'", df[treat_col].head().to_string())

        # Convertir a numérico
        df['target_treat'] = pd.to_numeric(df[treat_col], errors='coerce').fillna(0)

        print(f"   Distribución:")
        print(f"     • Buscaron tratamiento: {df['target_treat'].sum():4d} ({df['target_treat'].sum()/len(df)*100:5.1f}%)者に {df['target_treat'].sum():4d} ({(1-df['target_treat'].sum()/len(df))*100:5.1f}%)")

    return df

# ============================================
# EJECUCIÓN PRINCIPAL
# ============================================

if __name__ == "__main__":

    print("="*80)
    print("🚀 FASE 3: CÁLCULO DE ÍNDICES Y CORRELACIONES")
    print("="*80)

    # 1. Cargar datos
    df, response_map = load_clean_data()
    print(f"\n✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")

    # 2. Identificar columnas para cada índice
    support_cols, stigma_cols, openness_cols = identify_index_columns(df)

    print(f"\n📋 COLUMNAS IDENTIFICADAS (cargadas desde JSON):")
    print(f"   • Support Index: {len(support_cols)} columnas")
    for col in support_cols[:3]:
        print(f"     - {col[:70]}...")
    print(f"   • Stigma Index: {len(stigma_cols)} columnas")
    for col in stigma_cols[:3]:
        print(f"     - {col[:70]}...")
    print(f"   • Openness Index: {len(openness_cols)} columnas")
    for col in openness_cols[:3]:
        print(f"     - {col[:70]}...")

    # 3. Guardar diccionario de mapeo actualizado
    with open('response_map_complete.json', 'w') as f:
        json.dump({str(k): v for k, v in response_map.items()}, f, indent=2)

    # 4. Calcular correlaciones para cada índice
    support_pairs = calculate_spearman_correlations(df, support_cols, "MENTAL HEALTH SUPPORT INDEX")
    stigma_pairs = calculate_spearman_correlations(df, stigma_cols, "WORKPLACE STIGMA INDEX")
    openness_pairs = calculate_spearman_correlations(df, openness_cols, "ORGANIZATIONAL OPENNESS SCORE")

    # 5. Calcular los índices
    df = calculate_index_scores(df, support_cols, stigma_cols, openness_cols, response_map)

    # 6. Crear variables target
    df = create_target_variables(df)

    # 7. GUARDAR DATASET COMPLETO
    output_file = 'mental_health_with_indexes.csv'
    df.to_csv(output_file, index=False)
    print(f"\n💾 Dataset con índices guardado: '{output_file}'")
    print(f"   • Filas: {df.shape[0]}")
    print(f"   • Columnas: {df.shape[1]}")
    print(f"   • Nuevas columnas: support_index, stigma_index, openness_index, target_diag, target_treat")

    # 8. RESUMEN PARA EXCEL
    print("\n" + "="*80)
    print("📋 RESUMEN PARA PLANTILLA EXCEL")
    print("="*80)

    print("\n🔵 SUPPORT INDEX - TOP 5 CORRELACIONES:")
    for i, (col1, col2, val) in enumerate(support_pairs, 1):
        print(f"   {i}. {col1[:50]}...")
        print(f"      vs {col2[:50]}...")
        print(f"      (r = {val:.4f})")

    print("\n🔴 STIGMA INDEX - TOP 5 CORRELACIONES:")
    for i, (col1, col2, val) in enumerate(stigma_pairs, 1):
        print(f"   {i}. {col1[:50]}...")
        print(f"      vs {col2[:50]}...")
        print(f"      (r = {val:.4f})")

    print("\n🟢 OPENNESS INDEX - TOP 5 CORRELACIONES:")
    for i, (col1, col2, val) in enumerate(openness_pairs, 1):
        print(f"   {i}. {col1[:50]}...")
        print(f"      vs {col2[:50]}...")
        print(f"      (r = {val:.4f})")

    print("\n" + "="*80)
    print("✅ FASE 3 COMPLETADA - ÍNDICES Y CORRELACIONES LISTOS")
    print("="*80)

🚀 FASE 3: CÁLCULO DE ÍNDICES Y CORRELACIONES

✓ Datos cargados: 1433 filas, 105 columnas

Loading column categories from 'column_categories.json'...

📋 COLUMNAS IDENTIFICADAS (cargadas desde JSON):
   • Support Index: 40 columnas
     - Do you think that team members/co-workers would view you more negative...
     - Were you aware of the options for mental health care provided by your ...
     - Have you heard of or observed negative consequences for co-workers who...
   • Stigma Index: 18 columnas
     - Do you think that team members/co-workers would view you more negative...
     - If you have been diagnosed or treated for a mental health disorder, do...
     - How willing would you be to share with friends and family that you hav...
   • Openness Index: 11 columnas
     - If you have been diagnosed or treated for a mental health disorder, do...
     - Has your employer ever formally discussed mental health (for example, ...
     - Did your previous employers ever formally discuss m

In [9]:
# ============================================================
# STEP 6: CLUSTERING (K=3) — columnas reales del CSV
# Usa las mismas columnas que componen los 3 índices
# ============================================================

import pandas as pd
import numpy as np
import json
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

print("=" * 80)
print("FASE 6: CLUSTERING - PERFILES DE EMPLEADOS (K=3)")
print("=" * 80)

df = pd.read_csv('mental_health.csv')
print(f"\n✓ Datos cargados: {df.shape[0]} filas, {df.shape[1]} columnas")

INDEX_MAPS_CLUSTER = {
    'Does your employer provide mental health benefits as part of healthcare coverage?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5, 'Not eligible for coverage / N/A': 0},
    'Do you know the options for mental health care available under your employer-provided coverage?':
        {'Yes': 1, 'No': 0, 'I am not sure': 0.5},
    'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5},
    'Does your employer offer resources to learn more about mental health concerns and options for seeking help?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5},
    'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5},
    'Do you think that discussing a mental health disorder with your employer would have negative consequences?':
        {'Yes': 1, 'Maybe': 0.5, 'No': 0},
    'Do you think that discussing a physical health issue with your employer would have negative consequences?':
        {'Yes': 1, 'Maybe': 0.5, 'No': 0},
    'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?':
        {'Yes': 1, 'No': 0},
    'Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?':
        {'Yes, I experienced': 1, 'Yes, I observed': 0.75, 'Maybe/Not sure': 0.5, 'No': 0},
    'Do you think that discussing a mental health disorder with previous employers would have negative consequences?':
        {'Yes, all of them': 1, 'Some of them': 0.5, "I don't know": 0.5, 'None of them': 0},
    'Do you think that discussing a physical health issue with previous employers would have negative consequences?':
        {'Yes, all of them': 1, 'Some of them': 0.5, 'None of them': 0},
    'Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?':
        {'Yes, all of them': 1, 'Some of them': 0.5, 'None of them': 0},
    'Do you feel that being identified as a person with a mental health issue would hurt your career?':
        {'Yes, I think it would': 1, 'Yes, it has': 1, 'Maybe': 0.5,
         "No, I don't think it would": 0, 'No, it has not': 0},
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?':
        {'Yes, I think they would': 1, 'Yes, they do': 1, 'Maybe': 0.5,
         "No, I don't think they would": 0, 'No, they do not': 0},
    'Would you feel comfortable discussing a mental health disorder with your coworkers?':
        {'Yes': 1, 'Maybe': 0.5, 'No': 0},
    'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?':
        {'Yes': 1, 'Maybe': 0.5, 'No': 0},
    'Would you have been willing to discuss a mental health issue with your previous co-workers?':
        {'Yes, at all of my previous employers': 1, 'Some of my previous employers': 0.5,
         'No, at none of my previous employers': 0},
    'Would you have been willing to discuss a mental health issue with your direct supervisor(s)?':
        {'Yes, at all of my previous employers': 1, 'Some of my previous employers': 0.5,
         "I don't know": 0.5, 'No, at none of my previous employers': 0},
    'Do you feel that your employer takes mental health as seriously as physical health?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5},
    'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:':
        {'Very easy': 1, 'Somewhat easy': 0.75, 'Neither easy nor difficult': 0.5,
         "I don't know": 0.5, 'Somewhat difficult': 0.25, 'Very difficult': 0},
    'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?':
        {'Yes, they all did': 1, 'Some did': 0.5, "I don't know": 0.5, 'None did': 0},
    'Did you feel that your previous employers took mental health as seriously as physical health?':
        {'Yes, they all did': 1, 'Some did': 0.5, "I don't know": 0.5, 'None did': 0},
}

SUPPORT_COLS = [
    'Does your employer provide mental health benefits as part of healthcare coverage?',
    'Do you know the options for mental health care available under your employer-provided coverage?',
    'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
    'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
    'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
]
STIGMA_COLS = [
    'Do you think that discussing a mental health disorder with your employer would have negative consequences?',
    'Do you think that discussing a physical health issue with your employer would have negative consequences?',
    'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?',
    'Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?',
    'Do you think that discussing a mental health disorder with previous employers would have negative consequences?',
    'Do you think that discussing a physical health issue with previous employers would have negative consequences?',
    'Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?',
    'Do you feel that being identified as a person with a mental health issue would hurt your career?',
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?',
]
OPENNESS_COLS = [
    'Would you feel comfortable discussing a mental health disorder with your coworkers?',
    'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?',
    'Would you have been willing to discuss a mental health issue with your previous co-workers?',
    'Would you have been willing to discuss a mental health issue with your direct supervisor(s)?',
    'Do you feel that your employer takes mental health as seriously as physical health?',
    'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:',
    'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',
    'Did you feel that your previous employers took mental health as seriously as physical health?',
]
ALL_COLS = list(dict.fromkeys(SUPPORT_COLS + STIGMA_COLS + OPENNESS_COLS))

X = pd.DataFrame(index=df.index)
for col in ALL_COLS:
    enc = df[col].map(INDEX_MAPS_CLUSTER[col]) if col in INDEX_MAPS_CLUSTER \
          else pd.to_numeric(df[col], errors='coerce')
    X[col] = pd.to_numeric(enc, errors='coerce')

X_imp = pd.DataFrame(
    SimpleImputer(strategy='median').fit_transform(X),
    columns=X.columns, index=X.index
)
X_scaled = StandardScaler().fit_transform(X_imp)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(X_scaled)

print(f"\n📊 Distribución de clusters:")
for i in range(3):
    n = sum(df['cluster'] == i)
    print(f"   Cluster {i}: {n} ({n/len(df)*100:.1f}%)")

global_means  = X_imp.mean()
cm = X_imp.copy()
cm['cluster'] = df['cluster'].values
cluster_means = cm.groupby('cluster')[ALL_COLS].mean()

print("\n" + "=" * 65)
print("TOP 3 FEATURES POR CLUSTER (vs MEDIA GLOBAL)")
print("=" * 65)

top_features = {}
for cluster in range(3):
    diff = (cluster_means.loc[cluster] - global_means).abs()
    top3 = diff.nlargest(3).index.tolist()
    top_features[f'cluster_{cluster}'] = top3
    print(f"\n   Cluster {cluster}:")
    for feat in top3:
        cv = cluster_means.loc[cluster, feat]
        gv = global_means[feat]
        print(f"     {'▲' if cv>gv else '▼'} {feat[:72]}")
        print(f"       cluster={cv:.3f} vs global={gv:.3f} (diff={cv-gv:+.3f})")

# Verificación: siempre exactamente 9 valores (3 clusters × 3 features)
total_vals = sum(len(v) for v in top_features.values())
assert total_vals == 9, f"ERROR: top_features tiene {total_vals} valores, esperados 9"
print(f"\n✅ Verificación: {total_vals} valores (3×3=9 ✓)")

with open('cluster_top_features.json', 'w') as f_out:
    json.dump(top_features, f_out, indent=2)
print("💾 cluster_top_features.json guardado")
print("\n" + "=" * 80)
print("✅ FASE 6 COMPLETADA")
print("=" * 80)


FASE 6: CLUSTERING - PERFILES DE EMPLEADOS (K=3)

✓ Datos cargados: 1433 filas, 62 columnas

📊 Distribución de clusters:
   Cluster 0: 626 (43.7%)
   Cluster 1: 452 (31.5%)
   Cluster 2: 355 (24.8%)

TOP 3 FEATURES POR CLUSTER (vs MEDIA GLOBAL)

   Cluster 0:
     ▼ Has your employer ever formally discussed mental health (for example, as
       cluster=0.057 vs global=0.196 (diff=-0.140)
     ▼ Have you observed or experienced an unsupportive or badly handled respon
       cluster=0.282 vs global=0.406 (diff=-0.124)
     ▼ Did you hear of or observe negative consequences for co-workers with men
       cluster=0.087 vs global=0.198 (diff=-0.111)

   Cluster 1:
     ▲ Do you think that team members/co-workers would view you more negatively
       cluster=0.824 vs global=0.517 (diff=+0.307)
     ▲ Do you think that discussing a mental health disorder with your employer
       cluster=0.692 vs global=0.424 (diff=+0.268)
     ▲ Do you feel that being identified as a person with a mental hea

In [10]:
# ============================================================
# STEP 4: SUBMISSION — ÍNDICES, ML FEATURES, F1, CLUSTERING
# Lee mental_health.csv y submission_output_template.csv
# Requiere cluster_top_features.json (generado en STEP 6)
# ============================================================

import pandas as pd
import numpy as np
import json
from scipy import stats
from itertools import combinations
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# ── Carga única del dataset original ──────────────────────────
df_raw = pd.read_csv('mental_health.csv')
print(f"✓ Dataset cargado: {df_raw.shape[0]} filas, {df_raw.shape[1]} columnas")

# ── Encoding maps (definidos una sola vez) ────────────────────

INDEX_MAPS = {
    'Does your employer provide mental health benefits as part of healthcare coverage?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5, 'Not eligible for coverage / N/A': 0},
    'Do you know the options for mental health care available under your employer-provided coverage?':
        {'Yes': 1, 'No': 0, 'I am not sure': 0.5},
    'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5},
    'Does your employer offer resources to learn more about mental health concerns and options for seeking help?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5},
    'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5},
    'Do you think that discussing a mental health disorder with your employer would have negative consequences?':
        {'Yes': 1, 'Maybe': 0.5, 'No': 0},
    'Do you think that discussing a physical health issue with your employer would have negative consequences?':
        {'Yes': 1, 'Maybe': 0.5, 'No': 0},
    'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?':
        {'Yes': 1, 'No': 0},
    'Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?':
        {'Yes, I experienced': 1, 'Yes, I observed': 0.75, 'Maybe/Not sure': 0.5, 'No': 0},
    'Do you think that discussing a mental health disorder with previous employers would have negative consequences?':
        {'Yes, all of them': 1, 'Some of them': 0.5, "I don't know": 0.5, 'None of them': 0},
    'Do you think that discussing a physical health issue with previous employers would have negative consequences?':
        {'Yes, all of them': 1, 'Some of them': 0.5, 'None of them': 0},
    'Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?':
        {'Yes, all of them': 1, 'Some of them': 0.5, 'None of them': 0},
    'Do you feel that being identified as a person with a mental health issue would hurt your career?':
        {'Yes, I think it would': 1, 'Yes, it has': 1, 'Maybe': 0.5,
         "No, I don't think it would": 0, 'No, it has not': 0},
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?':
        {'Yes, I think they would': 1, 'Yes, they do': 1, 'Maybe': 0.5,
         "No, I don't think they would": 0, 'No, they do not': 0},
    'Would you feel comfortable discussing a mental health disorder with your coworkers?':
        {'Yes': 1, 'Maybe': 0.5, 'No': 0},
    'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?':
        {'Yes': 1, 'Maybe': 0.5, 'No': 0},
    'Would you have been willing to discuss a mental health issue with your previous co-workers?':
        {'Yes, at all of my previous employers': 1, 'Some of my previous employers': 0.5,
         'No, at none of my previous employers': 0},
    'Would you have been willing to discuss a mental health issue with your direct supervisor(s)?':
        {'Yes, at all of my previous employers': 1, 'Some of my previous employers': 0.5,
         "I don't know": 0.5, 'No, at none of my previous employers': 0},
    'Do you feel that your employer takes mental health as seriously as physical health?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5},
    'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:':        {'Very easy': 1, 'Somewhat easy': 0.75, 'Neither easy nor difficult': 0.5,
         "I don't know": 0.5, 'Somewhat difficult': 0.25, 'Very difficult': 0},
    'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?':        {'Yes, they all did': 1, 'Some did': 0.5, "I don't know": 0.5, 'None did': 0},
    'Did you feel that your previous employers took mental health as seriously as physical health?':        {'Yes, they all did': 1, 'Some did': 0.5, "I don't know": 0.5, 'None did': 0},
}

ORDINAL_MAPS_ML = {
    # Features workplace — sin leakage de historial clínico personal
    'Do you know the options for mental health care available under your employer-provided coverage?':
        {'Yes': 1, 'No': 0, 'I am not sure': 0.5},
    'Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?':
        {'Yes, I experienced': 1, 'Yes, I observed': 0.75, 'Maybe/Not sure': 0.5, 'No': 0},
    'Were you aware of the options for mental health care provided by your previous employers?':
        {'Yes, I was aware of all of them': 1, 'I was aware of some': 0.5,
         'No, I only became aware later': 0.25, 'N/A (not currently aware)': 0},
    'Do you think that discussing a mental health disorder with previous employers would have negative consequences?':
        {'Yes, all of them': 1, 'Some of them': 0.5, "I don't know": 0.5, 'None of them': 0},
    'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?':
        {'Yes, always': 1, 'Sometimes, if it comes up': 0.5,
         'No, because it would impact me negatively': 0,
         "No, because it doesn't matter": 0, 'Not applicable to me': np.nan},
    'Did you feel that your previous employers took mental health as seriously as physical health?':
        {'Yes, they all did': 1, 'Some did': 0.5, "I don't know": 0.5, 'None did': 0},
    'Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?':
        {'Yes, all of them': 1, 'Some of them': 0.5, 'None of them': 0},
    'Does your employer provide mental health benefits as part of healthcare coverage?':
        {'Yes': 1, 'No': 0, "I don't know": 0.5, 'Not eligible for coverage / N/A': 0},
    'If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?':
        {'Yes, always': 1, 'Sometimes, if it comes up': 0.5,
         'No, because it would impact me negatively': 0,
         "No, because it doesn't matter": 0, 'Not applicable to me': np.nan},
    'How willing would you be to share with friends and family that you have a mental illness?':
        {'Very open': 1, 'Somewhat open': 0.75, 'Neutral': 0.5, 'Somewhat not open': 0.25,
         'Not open at all': 0, 'Not applicable to me (I do not have a mental illness)': np.nan},
    'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:':
        {'Very easy': 1, 'Somewhat easy': 0.75, 'Neither easy nor difficult': 0.5,
         "I don't know": 0.5, 'Somewhat difficult': 0.25, 'Very difficult': 0},
    'If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?':
        {'Yes': 1, "I'm not sure": 0.5, 'No': 0},
    'Would you bring up a mental health issue with a potential employer in an interview?':
        {'Yes': 1, 'Maybe': 0.5, 'No': 0},
}

EXCLUDE_COLS = {
    # Targets
    'Do you currently have a mental health disorder?',
    'Have you ever sought treatment for a mental health issue from a mental health professional?',
    # Leakage — historial clínico personal (no workplace)
    'Have you been diagnosed with a mental health condition by a medical professional?',
    'Have you had a mental health disorder in the past?',
    'If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?',
    'If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?',
    'If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?',
    'Do you believe your productivity is ever affected by a mental health issue?',
    'Do you have a family history of mental illness?',
    'Do you know local or online resources to seek help for a mental health disorder?',
    # Texto libre
    'If maybe, what condition(s) do you believe you have?',
    'If so, what condition(s) were you diagnosed with?',
    'Why or why not?', 'Why or why not?.1',
    # Ubicación / demografía
    'What US state or territory do you live in?', 'What US state or territory do you work in?',
    'What country do you live in?', 'What country do you work in?',
    'What is your gender?', 'Which of the following best describes your work position?',
    'What is your age?',
}

SUPPORT_COLS = [
    'Does your employer provide mental health benefits as part of healthcare coverage?',
    'Do you know the options for mental health care available under your employer-provided coverage?',
    'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
    'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
    'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
]
STIGMA_COLS = [
    'Do you think that discussing a mental health disorder with your employer would have negative consequences?',
    'Do you think that discussing a physical health issue with your employer would have negative consequences?',
    'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?',
    'Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?',
    'Do you think that discussing a mental health disorder with previous employers would have negative consequences?',
    'Do you think that discussing a physical health issue with previous employers would have negative consequences?',
    'Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?',
    'Do you feel that being identified as a person with a mental health issue would hurt your career?',
    'Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?',
]
OPENNESS_COLS = [
    'Would you feel comfortable discussing a mental health disorder with your coworkers?',
    'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?',
    'Would you have been willing to discuss a mental health issue with your previous co-workers?',
    'Would you have been willing to discuss a mental health issue with your direct supervisor(s)?',
    'Do you feel that your employer takes mental health as seriously as physical health?',
    'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:',
    'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',
    'Did you feel that your previous employers took mental health as seriously as physical health?',
]

# ── Helpers ───────────────────────────────────────────────────

def calc_top5_pairs(df, cols, label):
    """Top 5 pares de columnas por correlación de Spearman."""
    encoded = {}
    for col in cols:
        enc = df[col].map(INDEX_MAPS[col]) if col in INDEX_MAPS else pd.to_numeric(df[col], errors='coerce')
        enc = pd.to_numeric(enc, errors='coerce')
        if enc.notna().sum() >= 100:
            encoded[col] = enc
    pairs = {}
    for c1, c2 in combinations(encoded.keys(), 2):
        mask = encoded[c1].notna() & encoded[c2].notna()
        if mask.sum() < 100:
            continue
        r, _ = stats.spearmanr(encoded[c1][mask], encoded[c2][mask])
        pairs[(c1, c2)] = abs(r)
    top5 = sorted(pairs.items(), key=lambda x: x[1], reverse=True)[:5]
    print(f'\n  📊 {label}:')
    for i, ((c1, c2), r) in enumerate(top5, 1):
        print(f'    {i}. r={r:.4f}  {c1[:45]} <-> {c2[:45]}')
    return [(c1, c2) for (c1, c2), _ in top5]

def get_top10_features(df, y):
    """Top 10 columnas originales por correlación de Spearman con el target."""
    results = {}
    for col in df.columns:
        if col in EXCLUDE_COLS:
            continue
        enc = df[col].map(ORDINAL_MAPS_ML[col]) if col in ORDINAL_MAPS_ML else pd.to_numeric(df[col], errors='coerce')
        enc = pd.to_numeric(enc, errors='coerce')
        mask = enc.notna() & y.notna()
        if mask.sum() < 100:
            continue
        r, _ = stats.spearmanr(enc[mask], y[mask])
        results[col] = abs(r)
    return [col for col, _ in sorted(results.items(), key=lambda x: x[1], reverse=True)[:10]]

def encode_for_model(df, cols):
    """Encodea columnas y aplica imputación por mediana."""
    X = pd.DataFrame(index=df.index)
    for col in cols:
        X[col] = df[col].map(ORDINAL_MAPS_ML[col]) if col in ORDINAL_MAPS_ML else pd.to_numeric(df[col], errors='coerce')
    imp = SimpleImputer(strategy='median')
    return pd.DataFrame(imp.fit_transform(X), columns=X.columns, index=X.index)

def run_cv(X, y, label):
    """Validación cruzada 5-fold con HistGradientBoosting."""
    model = HistGradientBoostingClassifier(
        max_iter=100, learning_rate=0.1, max_depth=3,
        random_state=42, class_weight='balanced'
    )
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    print(f'\n  Validación Cruzada — {label}:')
    for fold, (tr, val) in enumerate(cv.split(X, y), 1):
        model.fit(X.iloc[tr], y.iloc[tr])
        scores.append(f1_score(y.iloc[val], model.predict(X.iloc[val]), average='weighted'))
        print(f'    Fold {fold}: F1 = {scores[-1]:.4f}')
    mean = round(np.mean(scores), 4)
    print(f'    ✅ F1 = {mean:.4f} ± {np.std(scores):.4f}')
    return mean

# ── Targets ───────────────────────────────────────────────────
y1 = df_raw['Do you currently have a mental health disorder?'].map({'Yes': 1, 'No': 0, 'Maybe': 0}).fillna(0)
y2 = pd.to_numeric(df_raw['Have you ever sought treatment for a mental health issue from a mental health professional?'], errors='coerce').fillna(0)

# ── SECCIÓN 1: Top 5 pares por índice ─────────────────────────
print('\n' + '='*65)
print('SECCIÓN 1 — TOP 5 PARES POR ÍNDICE')
print('='*65)
support_pairs  = calc_top5_pairs(df_raw, SUPPORT_COLS,  'MENTAL HEALTH SUPPORT INDEX')
stigma_pairs   = calc_top5_pairs(df_raw, STIGMA_COLS,   'WORKPLACE STIGMA INDEX')
openness_pairs = calc_top5_pairs(df_raw, OPENNESS_COLS, 'ORGANIZATIONAL OPENNESS SCORE')

# ── SECCIÓN 2: Top 10 features por target ────────────────────
print('\n' + '='*65)
print('SECCIÓN 2 — TOP 10 FEATURES ORIGINALES POR TARGET')
print('='*65)
top10_diag  = get_top10_features(df_raw, y1)
top10_treat = get_top10_features(df_raw, y2)
print('\n  Target 1 — Diagnóstico:')
for i, c in enumerate(top10_diag, 1):
    print(f'    {i:2d}. {c[:72]}')
print('\n  Target 2 — Tratamiento:')
for i, c in enumerate(top10_treat, 1):
    print(f'    {i:2d}. {c[:72]}')

# ── SECCIÓN 3: F1 con validación cruzada real ─────────────────
print('\n' + '='*65)
print('SECCIÓN 3 — F1 SCORES CON VALIDACIÓN CRUZADA 5-FOLD')
print('='*65)
f1_diag  = run_cv(encode_for_model(df_raw, top10_diag),  y1, 'Diagnóstico')
f1_treat = run_cv(encode_for_model(df_raw, top10_treat), y2, 'Tratamiento')

# ── SECCIÓN 4: Clustering — leer desde JSON ───────────────────
print('\n' + '='*65)
print('SECCIÓN 4 — CLUSTERING (leído desde cluster_top_features.json)')
print('='*65)
try:
    with open('cluster_top_features.json') as f:
        cluster_top = json.load(f)
    print(f'  ✅ cluster_top_features.json cargado')
    for k, v in cluster_top.items():
        print(f'    {k}: {v}')
except FileNotFoundError:
    raise RuntimeError(
        "cluster_top_features.json no encontrado. "
        "Ejecuta primero la celda STEP 6 (Clustering)."
    )

# ── SECCIÓN 5: Generar CSV de submission ──────────────────────
print('\n' + '='*65)
print('SECCIÓN 5 — GENERANDO CSV DE SUBMISSION')
print('='*65)

with open('submission_output_template.csv', 'r', encoding='utf-8') as f:
    column_names = f.readline().strip().split(';')
column_names[0] = column_names[0].lstrip('\ufeff')

results = []
for c1, c2 in support_pairs:
    results += [c1, c2]
for c1, c2 in stigma_pairs:
    results += [c1, c2]
for c1, c2 in openness_pairs:
    results += [c1, c2]
results.extend(top10_diag)
results.extend(top10_treat)
results.append(f1_diag)
results.append(f1_treat)
for i in range(3):
    results.extend(cluster_top[f'cluster_{i}'])

assert len(results) == len(column_names), \
    f'ERROR: {len(results)} valores vs {len(column_names)} columnas'

submission_df = pd.DataFrame([results], columns=column_names)
submission_df.to_csv('submission_output_filled.csv', index=False, sep=';', encoding='utf-8')

print(f'\n  ✅ submission_output_filled.csv generado')
print(f'  📋 {len(column_names)} columnas | separador: ;')
print(f'  F1 Diagnóstico : {f1_diag}')
print(f'  F1 Tratamiento : {f1_treat}')


✓ Dataset cargado: 1433 filas, 62 columnas

SECCIÓN 1 — TOP 5 PARES POR ÍNDICE

  📊 MENTAL HEALTH SUPPORT INDEX:
    1. r=0.4605  Has your employer ever formally discussed men <-> Does your employer offer resources to learn m
    2. r=0.4386  Does your employer provide mental health bene <-> Do you know the options for mental health car
    3. r=0.3803  Does your employer provide mental health bene <-> Does your employer offer resources to learn m
    4. r=0.3056  Does your employer provide mental health bene <-> Is your anonymity protected if you choose to 
    5. r=0.2952  Does your employer offer resources to learn m <-> Is your anonymity protected if you choose to 

  📊 WORKPLACE STIGMA INDEX:
    1. r=0.5821  Do you feel that being identified as a person <-> Do you think that team members/co-workers wou
    2. r=0.5110  Do you think that discussing a mental health  <-> Do you think that team members/co-workers wou
    3. r=0.5086  Do you think that discussing a mental health  <-> 